In [2]:
import win32com.client as win32
import pandas as pd
import datetime as datetime
from datetime import date
from datetime import timedelta
from PIL import ImageGrab
import numpy as np
import os,sys


#Getting to the Root Folder
outlook=win32.Dispatch('outlook.application').GetNamespace("MAPI")
root_folder = outlook.Folders.Item('Your Email ID') 
s= root_folder.Folders['Inbox'].Folders['PRV']

# this is set to the current time
date_time = datetime.datetime.now()

#we can change timedelta's argument to whatever you want it to be
last30MinuteDateTime = datetime.datetime.now() - datetime.timedelta(days = 3)

# retrieve all emails in the inbox, then sort them from most recently received to oldest (False will give you the reverse). Not strictly necessary, but good to know if order matters for your search
messages = s.Items.Restrict("[Unread]=False")
messages.Sort("[ReceivedTime]", True)

last30MinuteMessages = messages.Restrict("[ReceivedTime] >= '" +last30MinuteDateTime.strftime('%m/%d/%Y %H:%M %p')+"'")

print ("Current time: "+date_time.strftime('%m/%d/%Y %H:%M %p'))

c=0
for message in last30MinuteMessages:
    print ("Subject :", message.subject)
    print ("Received Time :",message.ReceivedTime)
    for att in message.Attachments:
        try:
            att.SaveAsFile('File Location to save file' + '\\Incidents.xls')
            print("Attachement successfully Downloaded")
        except:
            print("Not successfull")
    c=c+1;

if(c>0):
    print ("The count of latest Dump from past 30 minutes ==",c)
    #getting the Dump File        
    file =r'File Location'
else:
    print("No recent Data")
    
    

#saving the output Reports
Rep= r'File Location'
Tic= r'File Location'

#Filtering the required columns oly
cols_to_use=['Incident Number','Description','Submit Date','Status','Priority',
             'Time To Breached In','Remaining Time To Be Breached','SLM_Status',
             'Assignee','Assigned Group',]

df = pd.read_excel(file,skiprows=5,index_col=False,usecols=cols_to_use)[cols_to_use]

#Removing the duplicate values
df = df.drop_duplicates(subset='Incident Number', keep="first")

#Finding the Aging Days
df['Submit Date'] = pd.to_datetime(df['Submit Date'])
df['Date']= df['Submit Date'].dt.date

df['Aging(Days)']=(pd.datetime.now().date()-df['Date']).astype(str)

df['Aging(Days)']= df['Aging(Days)'].str.replace('days 00:00:00.000000000', '')

#Delete Temperory Column
del df['Date']

#Rearrange Columns as needed
df= df[['Incident Number', 'Description', 'Submit Date', 'Aging(Days)', 'Status', 'Priority',
       'Time To Breached In', 'Remaining Time To Be Breached', 'SLM_Status',
       'Assignee', 'Assigned Group']]

#Renaming the Column heading as wasn't able to read
df.rename(columns={'Assigned Group': 'Assigned_Group'}, inplace=True)

df=df[df.Assigned_Group!= 'Ericsson MS Team']
df=df[df.Assigned_Group!= 'EMS_Siebel']
df=df[df.Assigned_Group!= 'EMS_EOC']
df=df[df.Assigned_Group!= 'EMS_ODI']
df=df[df.Assigned_Group!= 'EMS_Smart_Dealer']
df=df[df.Assigned_Group!= 'EMS_Network_LB']
df=df[df.Assigned_Group!= 'EMS_SND']
df=df[df.Assigned_Group!= 'BO_IN SDP']
df=df[df.Assigned_Group!= 'EMS_EOC']
df=df[df.Assigned_Group!= 'TCC- Semati']
df=df[df.Assigned_Group!= 'IT NOC']


df = df.replace(np.nan, '', regex=True)

#making Excel with the priority Tickets
df2 = df.groupby(['Assigned_Group','Priority']).size().unstack()
print(df2)

#Finding Total
df2['Grand Total'] = df2.sum(axis=1)
df2.loc['Grand Total'] = df2.sum()

#making final Excel file
df2.reset_index(inplace=True)
writer = pd.ExcelWriter(Rep, engine='xlsxwriter')

df2.to_excel(writer, sheet_name='Report', index=False)
workbook = writer.book
worksheet = writer.sheets['Report']
#worksheet = workbook.add_worksheet()
bold = workbook.add_format({'bold': 1})


chart = workbook.add_chart({'type': 'column'})

# Configure the series.
chart.add_series({
    'name':       '=Report!$B$1',
    'categories': '=Report!$A$2:$A$'+str(len(df2['Assigned_Group'])+1),
    'values':     '=Report!$B$2:$B$'+str(len(df2[df2.columns[1]])+1),
    'data_labels': {'value': True}
    })
   
chart.add_series({
    'name':       '=Report!$C$1',
    'categories': '=Report!$A$2:$A$'+str(len(df2['Assigned_Group'])+1),
    'values':     '=Report!$C$2:$C$'+str(len(df2[df2.columns[1]])+1),
    'data_labels': {'value': True},
    })

chart.add_series({
    'name':       '=Report!$D$1',
    'categories': '=Report!$A$2:$A$'+str(len(df2['Assigned_Group'])+1),
    'values':     '=Report!$D$2:$D$'+str(len(df2[df2.columns[1]])+1),
    'data_labels': {'value': True}
    })

chart.add_series({
    'name':       '=Report!$E$1',
    'categories': '=Report!$A$2:$A$'+str(len(df2['Assigned_Group'])+1),
    'values':     '=Report!$E$2:$E$'+str(len(df2[df2.columns[1]])+1),
    'data_labels': {'value': True},
    })

# Add a chart title. 
chart.set_title({'name': 'Open TT Status'}) 
chart.set_x_axis({'name': 'Teams'})
chart.set_y_axis({'name': 'TT Total'})

# Set a data table on the X-Axis with the legend keys shown
chart.set_table({'show_keys': True})

# Hide the chart legend since the keys are shown on the data table
chart.set_legend({'position': 'None'})
# Set the plot size
chart.set_size({'width':1400, 'height': 500})
chart.set_style(42)
# Insert the chart into the worksheet (with an offset)
worksheet.insert_chart('G2', chart, {'x_offset': 35, 'y_offset': 10})
workbook.close()

#extracting image from excel into html body
excel = win32.Dispatch("Excel.Application")
workbook = excel.Workbooks.Open(r'File Location',False, False,None)
wb_folder = workbook.Path
print(wb_folder)

image_no = 0

for sheet in workbook.Worksheets:
    for n, shape in enumerate(sheet.Shapes):
        if shape.Name.startswith("Chart"):
            # Some debug output for console
            image_no += 1
            print ("---- Image No. %07i ----" % image_no)

            # Sequence number the pictures, if there's more than one
            num = "" if n == 0 else "_%03i" % n

            filename = sheet.Name+num+".jpg"
            file_path = os.path.join (wb_folder, filename)

            #print "Saving as %s" % file_path    # Debug output

            shape.Copy() # Copies from Excel to Windows clipboard

            # Use PIL (python imaging library) to save from Windows clipboard
            # to a file
            image = ImageGrab.grabclipboard()
            image.save(file_path,'jpeg')

excel.DisplayAlerts = False
excel.Quit()
      
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)

img = r'Image location.jpg'
mail.Attachments.Add(img)


mail.Subject = 'Open TT Status'
body = "<h4>Hi All,<br><br>Please find the open TT status below :</h4><br><img src='cid:Report.jpg'><br><br><br>Raw Data :<br><br>"+df.to_html(index=False)+"<br>BR//<br><br>";
mail.HTMLBody = (body)
mail.To = 'Receivers mail'
#mail.bcc
#mail.cc
mail.send()

Current time: 11/22/2019 13:41 PM
Subject : FW: IncidentsReceived Time : 2019-11-22 11:09:59+00:00
Attachement successfully Downloaded
The count of latest Dump from past 30 minutes == 1
Priority                   High    Low  Medium
Assigned_Group                                
CCBS- Charging              NaN    3.0     3.0
CRM analytics & Reporting   NaN    NaN     1.0
Customer Care & Billing     NaN   19.0     NaN
EMA Provisioning            NaN    NaN     1.0
EMM Offline Mediation       NaN    NaN     1.0
EMS DB Support              NaN    1.0     2.0
EMS E-Business Support      NaN   10.0     4.0
EMS SFA Support             NaN    NaN     1.0
EMS Sun Support            17.0  129.0    15.0
IN                          NaN    2.0     3.0
PEGA                        NaN    5.0     NaN
Provisioning                NaN    1.0     NaN
Zain Plus (LMS)             NaN    NaN     1.0
File Location---- Image No. 0000001 ----


TypeError: 'bool' object is not callable